<a href="https://colab.research.google.com/github/Coralain10/wv71_tf_202010279_202017572_201319550/blob/main/Trabajo_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lista de adyacencia entre nodos vecinos

In [1]:
'''
56 57 58 59 60 61 62 63
48 49 50 51 52 53 54 55
40 41 42 43 44 45 46 47
32 33 34 35 36 37 38 39
24 25 26 27 28 29 30 31
16 17 18 19 20 21 22 23
 8  9 10 11 12 13 14 15
 0  1  2  3  4  5  6  7

xy
07 17 27 37 47 57 67 77
06 16 26 36 46 56 66 76
05 15 25 35 45 55 65 75
04 14 24 34 44 54 64 74
03 13 23 33 43 53 63 73
02 12 22 32 42 52 62 72
01 11 21 31 41 51 61 71
00 10 20 30 40 50 60 70

32 => 3+2*8
ngrafo = y*ancho + x

00 | 01 10 => 0 | 1 8
0+1<n 0-1>0 0+ancho<n 0-ancho>0 

32 | 33 31 22 42 => 19 | 27 11 18 20
19+1<n 19-1>0 19+ancho<n 19-ancho>0 

77 | 76 67 => 63 | 55 62
63+1<n 63-1>0 63+ancho<n 63-ancho>0 
'''
''

''

## Función para unir nodos vecinos
* Une los vecinos de abajo, izquierda, derecha y arriba
* Los nodos se cuentan desde la esquina inferior izquierda hacia la esquina superior derecha.
* Se leen de izquierda a derecha y de abajo a arriba.

In [2]:
def setGraphByWH(w,h): # width height
  n = w*h
  G = [ [] for _ in range(n) ]
  for i, u in enumerate(G):
    if i - w >= 0 : u.append(i - w)
    if i - 1 >= 0 : u.append(i - 1)
    if i + 1  < n : u.append(i + 1)
    if i + w  < n : u.append(i + w)
  return G

def setGraphByWH2(w,h): #v2 # width height
  n = w*h
  G = []
  for i in range(n):
    u = []
    if i - w >= 0 : u.append(i - w)
    if i - 1 >= 0 : u.append(i - 1)
    if i + 1  < n : u.append(i + 1)
    if i + w  < n : u.append(i + w)
    G.append(u)
  return G

In [3]:
#%timeit setGraphByWH(7000,7000)
#%timeit setGraphByWH2(7000,7000) #7000 x 7100 no

side = 1000 #ancho y alto de la ciudad
G = setGraphByWH(side,side)

In [4]:
fileG = open("lista_adyacencia.csv","w")
for u in G:
  fileG.write(str(u)[1:-1] + '\n')
fileG.close() #to change file access modes

## Funcion para diferenciar los nodos (v1)
* Todos los nodos inician con x, y 
* Los nodos de distribución terminan en D
* Los nodos de entrega terminan en E

In [5]:
# width height delivery_points[(x,y)], distribution_points[(x,y)] 
# Solo modificar los labels
def setLabelsByPoints(G,deps,dips,w=None):
  n = len(G)
  if w == None: w = int(n**(1/2)) #10 | 2,1
  labels = [f'{i%w},{i//w}' for i in range(n)] #x,y
  for v in deps:
    x, y = v[0], v[1]
    pos = y*w + x
    labels[pos] = f'{x},{y} E'
  for v in dips:
    x, y = v[0], v[1]
    pos = y*w + x
    labels[pos] = f'{x},{y} D'
  return labels

## Funcion para diferenciar los nodos (v2)
* **w** es el ancho de la ciudad.
* La posición **i** del nodo en x,y es $i = y \cdot w + x$ .
* La posición **(x,y)** del nodo i se encuentra en $x = i\%w $ y en $y = i//w$.
* Solo se consideran los nodos de distribución y de entrega.

In [6]:
def getNodesByXYs(xyl, w): #xylist [(x,y),(x,y),...]
  return [xy[1]*w + xy[0] for xy in xyl]

def getXYsByNodes(nl, w):
  return [ (i%w , i//w) for i in nl ]

# Agregar la información de los datasets
* [x] Agregar la información de dataset de almacenes.csv.
* [x] Agregar la información de dataset de puntos_entrega.csv.

In [7]:
!wget https://raw.githubusercontent.com/Coralain10/wv71_tf_202010279_202017572_201319550/main/dataset/almacenes.csv -P dataset/
!wget https://raw.githubusercontent.com/Coralain10/wv71_tf_202010279_202017572_201319550/main/dataset/puntos_entrega.csv -P dataset/

--2021-11-25 16:37:18--  https://raw.githubusercontent.com/Coralain10/wv71_tf_202010279_202017572_201319550/main/dataset/almacenes.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 695 [text/plain]
Saving to: ‘dataset/almacenes.csv’

almacenes.csv       100%[===================>]     695  --.-KB/s    in 0s      

2021-11-25 16:37:19 (43.5 MB/s) - ‘dataset/almacenes.csv’ saved [695/695]

--2021-11-25 16:37:19--  https://raw.githubusercontent.com/Coralain10/wv71_tf_202010279_202017572_201319550/main/dataset/puntos_entrega.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP reques

In [8]:
def fileXYtoLst(fname):
  xys = []
  with open(fname) as f:
    for line in f:
      xys.append( tuple( map(int,line.strip().split(','))) )
  return xys

almacenes_xys = fileXYtoLst("dataset/almacenes.csv")
entrega_xys = fileXYtoLst("dataset/puntos_entrega.csv")

In [9]:
almacenes_nodes = getNodesByXYs(almacenes_xys, side)
entrega_nodes = getNodesByXYs(entrega_xys, side)

In [10]:
'''import matplotlib.pyplot as plt
plt.figure(figsize=(50,50))
plt.grid(color='#AAAAAA', linestyle='-', linewidth=1)
plt.scatter( *zip(*almacenes_xys), s=100, color="#ff9654", label="Puntos de distibucion")
plt.scatter( *zip(*entrega_xys), s=100, color="#34c0eb", label="Puntos de Entrega")
plt.xlabel("X")
plt.ylabel("Y")
for xy in almacenes_xys:
  plt.annotate(None,(xy[0], xy[1]))
for xy in entrega_xys:
  plt.annotate(None,(xy[0], xy[1]))
plt.legend()
plt.xlim([0, side])
plt.ylim([0, side])
plt.show()'''

'import matplotlib.pyplot as plt\nplt.figure(figsize=(50,50))\nplt.grid(color=\'#AAAAAA\', linestyle=\'-\', linewidth=1)\nplt.scatter( *zip(*almacenes_xys), s=100, color="#ff9654", label="Puntos de distibucion")\nplt.scatter( *zip(*entrega_xys), s=100, color="#34c0eb", label="Puntos de Entrega")\nplt.xlabel("X")\nplt.ylabel("Y")\nfor xy in almacenes_xys:\n  plt.annotate(None,(xy[0], xy[1]))\nfor xy in entrega_xys:\n  plt.annotate(None,(xy[0], xy[1]))\nplt.legend()\nplt.xlim([0, side])\nplt.ylim([0, side])\nplt.show()'

# Algoritmo de Dijkstra

In [11]:
import math
import heapq as hq
import numpy as np

In [16]:
def dijkstra(G, s):
  n = len(G)
  visited = [False]*n
  path = [None]*n
  cost = [math.inf]*n
  cost[s] = 0
  queue = [(0, s)]
  while queue:
    g_u, u = hq.heappop(queue)
    if not visited[u]:
      visited[u] = True
      for v in G[u]:
        f = g_u + 1
        if f < cost[v]:
          cost[v] = f
          path[v] = u
          hq.heappush(queue, (f, v))

  return path, cost

In [17]:
path, cost = dijkstra(G, 0)

## Dijkstra para cada punto de distribucion (v1)

In [18]:
def dijkstrapd(dis, entrega, G):
  res=[] 
  def dijkstrav2(G, s):
    n = len(G)
    visited = [False]*n
    path = [None]*n
    cost = [math.inf]*n
    cost[s] = 0
    queue = [(0, s)]
    while queue:
      g_u, u = hq.heappop(queue)
      if  not visited[u]: #
        visited[u] = True
        for v in G[u]:
          f = g_u + 1
          if f < cost[v]:
            cost[v] = f
            path[v] = u
            hq.heappush(queue, (f, v))
    return path, cost

  for i in dis:
    l1, l2 = dijkstrav2(G,i)
    for j in entrega:
      k = j
      p = [j]
      c = l2[j]
      while k!=i:
        k=l1[k]
        p.append(k)
      p.reverse()
      res.append((i,j,c,p)) #formato de la lista resultante: (nodo de distribucion i, nodo de entrega j, cost, path)
  return res

In [19]:
djpd = dijkstrapd(almacenes_nodes, entrega_nodes,G)

In [21]:
def showDj1(l:list, qv=10, qh=10):
  qv = min( len(l), qv )
  qh = min( len(l[0][-1]), qh )
  for i in range(qv):
    print( l[i][0], l[i][1], l[i][2], 
          str(l[i][3][:qh])[:-1], '...]', sep = ',' )
  print('...')
showDj1(djpd)

891193,622283,359,[891193, 890193, 889193, 888193, 887193, 886193, 885193, 884193, 883193, 882193,...]
891193,274440,864,[891193, 890193, 889193, 888193, 887193, 886193, 885193, 884193, 883193, 882193,...]
891193,893469,278,[891193, 891194, 891195, 891196, 891197, 891198, 891199, 891200, 891201, 891202,...]
891193,337414,775,[891193, 890193, 889193, 888193, 887193, 886193, 885193, 884193, 883193, 882193,...]
891193,358823,902,[891193, 890193, 889193, 888193, 887193, 886193, 885193, 884193, 883193, 882193,...]
891193,847001,236,[891193, 890193, 889193, 888193, 887193, 886193, 885193, 884193, 883193, 882193,...]
891193,430504,772,[891193, 890193, 889193, 888193, 887193, 886193, 885193, 884193, 883193, 882193,...]
891193,166465,997,[891193, 890193, 889193, 888193, 887193, 886193, 885193, 884193, 883193, 882193,...]
891193,758724,601,[891193, 890193, 889193, 888193, 887193, 886193, 885193, 884193, 883193, 882193,...]
891193,878502,322,[891193, 890193, 889193, 888193, 887193, 886193, 885193

In [46]:
def distancia(p1, p2):
  x1, y1 = p1
  x2, y2 = p2
  return math.sqrt((x2-x1)**2+(y2-y1)**2)

def getnodo(p, side):
  x, y = p
  return y*side + x

In [55]:
def asignacion(dist, ent):
  res = [] # [[dist1, emt1, ent2, entn],[dist2, ent1, ent2, entn],...]
  for i in dist:
    l = [getnodo(i,side)]
    for j in ent:
      if distancia(i,j)<=50:
        l.append(getnodo(j,side))
    res.append(l)
  return res


In [48]:
dist = fileXYtoLst("dataset/almacenes.csv")
ent = fileXYtoLst("dataset/puntos_entrega.csv")

In [83]:
l = asignacion(dist,ent)

In [77]:
def separacion(l):
  l1 = []
  l2 = []
  l3 = []
  for i in l:
    if len(i)<20:
      l1.append(i)
    elif len(i)>=20 and len(i)<25:
      l2.append(i)
    else:
      l3.append(i)
  return l1,l2,l3

In [78]:
s, c, j = separacion(l)